In [1]:
import pandas as pd
import numpy as np


In [28]:
data = pd.read_csv('data/laptops.csv')

In [ ]:
data['Final Price'].sort_values(ascending=False).head(5)

In [ ]:
max(data[data['Brand'] == 'Dell']['Final Price'])

In [ ]:
# Assuming df is your DataFrame
# Step 1: Calculate the median value of the Screen column
median_before = data['Screen'].median()

# Step 2: Calculate the most frequent value (mode) of the Screen column
mode_value = data['Screen'].mode()[0]

# Step 3: Use the fillna method to fill the missing values in the Screen column with the most frequent value
data['Screen'].fillna(mode_value, inplace=True)

# Step 4: Calculate the median value of the Screen column again
median_after = data['Screen'].median()

# Step 5: Compare the median values before and after filling the missing values
if median_before == median_after:
    print("No")
else:
    print("Yes")

In [ ]:
# Step 1: Select all the "Innjoo" laptops from the dataset
innjoo_laptops = data[data['Brand'] == 'Innjoo']

# Step 2: Select only columns RAM, Storage, Screen
selected_columns = innjoo_laptops[['RAM', 'Storage', 'Screen']]

# Step 3: Get the underlying NumPy array. Let's call it X
X = selected_columns.to_numpy()

# Step 4: Compute matrix-matrix multiplication between the transpose of X and X. To get the transpose, use X.T. Let's call the result XTX
XTX = X.T @ X

# Step 5: Compute the inverse of XTX
XTX_inv = np.linalg.inv(XTX)

# Step 6: Create an array y with values [1100, 1300, 800, 900, 1000, 1100]
y = np.array([1100, 1300, 800, 900, 1000, 1100])

# Step 7: Multiply the inverse of XTX with the transpose of X, and then multiply the result by y. Call the result w
w = XTX_inv @ X.T @ y

# Step 8: Compute the sum of all the elements of the result
sum_of_elements = np.sum(w)

print("Sum of all the elements of the result:", sum_of_elements)
